<a href="https://colab.research.google.com/github/mehdi780311/Innovation-Evaluation/blob/main/Thesis_Analyzer_with_Streamlit_and_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import streamlit as st
import pandas as pd
import requests
import io
import time

# --------------------------------------------------------------------------
# تابع برای ارسال درخواست به API و دریافت تحلیل
# --------------------------------------------------------------------------
def analyze_thesis_with_avvalai(title, abstract, api_key):
    """
    این تابع عنوان و چکیده را به همراه یک پرامپت ساختاریافته به API ارسال می‌کند
    و تحلیل مدل هوش مصنوعی را باز می‌گرداند.
    """
    # URL اندپوینت مدل‌های چت در avval.ai
    api_url = "https://api.avalapis.ir/v1"

    # هدرهای لازم برای احراز هویت
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # پرامپت (دستور) دقیق برای مدل هوش مصنوعی
    # این بخش مهم‌ترین قسمت است و به مدل می‌گوید دقیقاً چه کاری انجام دهد.
    prompt = f"""
    **دستورالعمل برای مدل هوش مصنوعی:**
    شما یک تحلیلگر متخصص در زمینه تجاری‌سازی تحقیقات دارویی و بیوتکنولوژی هستید. لطفاً عنوان و چکیده پایان‌نامه زیر را با دقت تحلیل کرده و به سه سوال زیر به صورت مجزا، واضح و شماره‌گذاری شده پاسخ دهید. برای هر بخش یک نمره از 1 تا 10 نیز اختصاص دهید.

    **عنوان پایان‌نامه:** "{title}"
    **چکیده پایان‌نامه:** "{abstract}"

    ---

    **1. پتانسیل تجاری‌سازی (Commercialization Potential):**
    آیا این تحقیق می‌تواند به یک محصول نهایی (دارو، کیت تشخیص، فرمولاسیون جدید) یا یک سرویس قابل فروش تبدیل شود؟ بازار هدف آن چیست؟ موانع اصلی برای ورود به بازار کدامند؟ (نمره از 1 تا 10)

    **2. سطح نوآوری (Innovation Level):**
    آیا ایده مطرح شده در این تحقیق جدید و خلاقانه است؟ چه شکافی از دانش را پر می‌کند؟ در مقایسه با روش‌ها و درمان‌های موجود چقدر نوآورانه محسوب می‌شود؟ (نمره از 1 تا 10)

    **3. پتانسیل ارزش‌آفرینی (Value Creation Potential):**
    این تحقیق چه ارزشی برای بیماران، جامعه علمی یا صنعت داروسازی ایجاد می‌کند؟ آیا می‌تواند به طور بالقوه هزینه‌های درمان را کاهش دهد، کیفیت زندگی بیماران را بهبود بخشد یا فرآیندهای داروسازی را بهینه‌تر کند؟ (نمره از 1 تا 10)
    """

    # بدنه درخواست (Payload)
    data = {
        "model": "avvalai-experimental",  # نام مدل را بر اساس مستندات avval.ai انتخاب کنید
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    # تلاش برای ارسال درخواست با مدیریت خطا
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.post(api_url, headers=headers, json=data, timeout=60)
            response.raise_for_status()  # بررسی خطاهای HTTP مانند 4xx یا 5xx
            result = response.json()
            analysis_text = result['choices'][0]['message']['content']
            return analysis_text.strip()
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                time.sleep(2) # Wait for 2 seconds before retrying
                continue
            else:
                st.error(f"خطا در ارتباط با API پس از {max_retries} تلاش: {e}")
                return f"خطا در تحلیل: {e}"
        except (KeyError, IndexError):
            st.error(f"پاسخ دریافت شده از API فرمت مورد انتظار را ندارد. پاسخ: {response.text}")
            return "خطا: فرمت پاسخ API نامعتبر است."
    return "خطا: تحلیل پس از چندین تلاش ناموفق بود."


# --------------------------------------------------------------------------
# رابط کاربری Streamlit
# --------------------------------------------------------------------------
st.set_page_config(layout="wide", page_title="تحلیلگر پایان‌نامه داروسازی")

# استایل سفارشی برای ظاهر بهتر
st.markdown("""
<style>
    .stButton>button {
        background-color: #4CAF50;
        color: white;
        border-radius: 12px;
        padding: 10px 24px;
    }
    .stDownloadButton>button {
        background-color: #008CBA;
        color: white;
        border-radius: 12px;
        padding: 10px 24px;
    }
</style>
""", unsafe_allow_html=True)


st.title("🔬 تحلیلگر هوشمند پایان‌نامه‌های داروسازی")
st.markdown("این ابزار با استفاده از هوش مصنوعی `avval.ai`، پتانسیل **تجاری‌سازی، نوآوری و ارزش‌آفرینی** پایان‌نامه‌ها را تحلیل می‌کند.")

# دریافت کلید API در نوار کناری
st.sidebar.header("تنظیمات")
api_key = st.sidebar.text_input("🔑 کلید API خود را از سایت avval.ai وارد کنید:", type="password", help="کلید API شما برای امنیت، ذخیره نخواهد شد.")

st.sidebar.markdown("---")
st.sidebar.info("فایل اکسل ورودی باید شامل دو ستون با نام‌های دقیق `Title` و `Abstract` باشد.")

# بخش بارگذاری فایل
uploaded_file = st.file_uploader("📂 فایل اکسل خود را اینجا بارگذاری کنید:", type=["xlsx"])

if uploaded_file is not None:
    try:
        df = pd.read_excel(uploaded_file)
        st.subheader("پیش‌نمایش داده‌های ورودی:")
        st.dataframe(df.head())

        # بررسی وجود ستون‌های الزامی
        if 'Title' not in df.columns or 'Abstract' not in df.columns:
            st.error("خطا: فایل اکسل باید حتماً شامل ستون‌های 'Title' و 'Abstract' باشد.")
        elif st.button("🚀 شروع تحلیل"):
            if not api_key:
                st.warning("لطفاً کلید API خود را در نوار کناری وارد کنید.")
            else:
                analyses = []
                total_rows = len(df)
                progress_bar = st.progress(0)
                status_text = st.empty()

                # حلقه برای تحلیل تک تک ردیف‌ها
                for index, row in df.iterrows():
                    title = str(row.get('Title', ''))
                    abstract = str(row.get('Abstract', ''))

                    # اطمینان از اینکه داده‌ها خالی نیستند
                    if title and abstract:
                        analysis_result = analyze_thesis_with_avvalai(title, abstract, api_key)
                        analyses.append(analysis_result)
                    else:
                        analyses.append("خطا: عنوان یا چکیده در این ردیف خالی است.")

                    # به‌روزرسانی نوار پیشرفت
                    progress_value = (index + 1) / total_rows
                    progress_bar.progress(progress_value)
                    status_text.text(f"در حال تحلیل ردیف {index + 1} از {total_rows}...")

                status_text.success("✅ تحلیل همه موارد با موفقیت انجام شد!")

                # اضافه کردن ستون جدید به دیتافریم
                df_results = df.copy()
                df_results['AI_Analysis'] = analyses

                st.subheader("نتایج تحلیل:")
                st.dataframe(df_results)

                # آماده‌سازی فایل اکسل برای دانلود
                output = io.BytesIO()
                with pd.ExcelWriter(output, engine='openpyxl') as writer:
                    df_results.to_excel(writer, index=False, sheet_name='Analysis_Results')

                excel_data = output.getvalue()

                st.download_button(
                    label="📥 دانلود نتایج در قالب فایل اکسل",
                    data=excel_data,
                    file_name="thesis_analysis_results.xlsx",
                    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
                )

    except Exception as e:
        st.error(f"یک خطا در پردازش فایل رخ داد: {e}")

ModuleNotFoundError: No module named 'streamlit'